# Laboratorio 6
Francisco Castillo - 21562

Diego Lemus - 21

## Task 1
### ¿Qué es Prioritized Sweeping para ambientes determinísticos?
Es una técnica de planeamiento que permite acelerar la propagación de los valores de los estados sin necesidad de actualizarlos todos uniformemente. En lugar de recorrer cada estado del espacio, se utiliza una cola de prioridades en la que se ordenan los estados según la magnitud del cambio que producen en sus valores. Cuando un estado se actualiza significativamente, los estados predecesores que llevan a él se incorporan a la cola para ser actualizados después. De este modo, los cambios en el valor de un estado se propagan hacia atrás de manera más eficiente, evitando cálculos innecesarios y enfocándose en las partes del espacio de estados donde realmente importa. En ambientes determinísticos esto resulta especialmente potente, ya que cada acción tiene un único resultado y la propagación de los valores puede seguir trayectorias claras.

### ¿Qué es Trajectory Sampling?
Es una técnica que evita la necesidad de explorar todo el árbol de estados y acciones posibles durante el planeamiento. En lugar de un análisis exhaustivo, el método simula trayectorias completas o episodios ficticios siguiendo la política actual y va actualizando los valores de los estados y acciones visitados en ese recorrido.

### ¿Qué es Upper Confidence Bounds para Árboles (UCT)?
Es una estrategia de selección que resuelve el dilema entre exploración y explotación. Para cada acción, combina el valor promedio estimado de los resultados obtenidos hasta el momento con un término de confianza que favorece aquellas opciones que han sido probadas pocas veces. De esta manera, el algoritmo asegura que no se quede únicamente con las ramas que parecen mejores al inicio, sino que también explore alternativas menos visitadas que podrían resultar óptimas.